In [1]:
import os
from glob import glob
import copy
from datetime import datetime
import distutils.dir_util


import numpy as np

from keras.layers import Dense
from keras.models import Model
from keras.applications.vgg16 import VGG16

from keras.optimizers import Adam
from keras import backend as K

from keras.callbacks import CSVLogger
from keras.preprocessing.image import ImageDataGenerator

Using TensorFlow backend.


In [2]:
DATA_ROOT_DIR = "/home/ubuntu/data/dogs-vs-cats/"
TEST_DIR = DATA_ROOT_DIR + '/test/'
RESULTS_DIR = DATA_ROOT_DIR + '/results/'

SAMPLE_ROOT_DIR = DATA_ROOT_DIR + "/sample/"
train_dir = SAMPLE_ROOT_DIR + '/train/'
valid_dir = SAMPLE_ROOT_DIR + '/valid/'
#train_dir = DATA_ROOT_DIR + '/train/'
#valid_dir = DATA_ROOT_DIR + '/valid/'

# 1. Prepare Data (create validation/sample sets, rearrange images by class dir)

In [5]:
%cd ~/data/dogs-vs-cats/
%mkdir valid
%mkdir results
%mkdir -p sample/train
%mkdir -p sample/valid
%mkdir -p test/unknown

/home/ubuntu/data/dogs-vs-cats


In [24]:
#%unzip test1.zip
#%unzip train1.zip
#%rm *.zip

In [31]:
# Create Validation
num_valid_one_class = 1000

%cd "train"
g = glob('cat*.jpg')
shuf = np.random.permutation(g)
for i in range(num_valid_one_class): os.rename(shuf[i], DATA_ROOT_DIR+'/valid/' + shuf[i])
    
g = glob('dog*.jpg')
shuf = np.random.permutation(g)
for i in range(num_valid_one_class): os.rename(shuf[i], DATA_ROOT_DIR+'/valid/' + shuf[i])

In [32]:
# Create Sample
num_sample_train = 200
num_sample_valid = 50


g = glob('*.jpg')
shuf = np.random.permutation(g)
for i in range(num_sample_train): copyfile(shuf[i], DATA_ROOT_DIR+'/sample/train/' + shuf[i])

%cd $DATA_ROOT_DIR/valid    
g = glob('*.jpg')
shuf = np.random.permutation(g)
for i in range(num_sample_valid): copyfile(shuf[i], DATA_ROOT_DIR+'/sample/valid/' + shuf[i])

/home/ubuntu/data/dogs-vs-cats/valid


In [33]:
# Rearrange images by class dir
%cd $DATA_ROOT_DIR/sample/train
%mkdir cats
%mkdir dogs
%mv cat.*.jpg cats/
%mv dog.*.jpg dogs/

%cd $DATA_ROOT_DIR/sample/valid
%mkdir cats
%mkdir dogs
%mv cat.*.jpg cats/
%mv dog.*.jpg dogs/

%cd $DATA_ROOT_DIR/valid
%mkdir cats
%mkdir dogs
%mv cat.*.jpg cats/
%mv dog.*.jpg dogs/

%cd $DATA_ROOT_DIR/train
%mkdir cats
%mkdir dogs
%mv cat.*.jpg cats/
%mv dog.*.jpg dogs/

/home/ubuntu/data/dogs-vs-cats/sample/train
/home/ubuntu/data/dogs-vs-cats/sample/valid
/home/ubuntu/data/dogs-vs-cats/valid
/home/ubuntu/data/dogs-vs-cats/train


In [36]:
# Create single 'unknown' class for test set
%cd $DATA_ROOT_DIR/test1
%mv *.jpg $DATA_ROOT_DIR/test/unknown/

%rmdir $DATA_ROOT_DIR/test1

/home/ubuntu/data/dogs-vs-cats/test1


# 2. Finetune model

In [3]:
def get_batches(path,
                gen=ImageDataGenerator(),
                shuffle=True, batch_size=64,
                class_mode='categorical'):
    return gen.flow_from_directory(path, 
                                   target_size=(224,224),
                                   class_mode=class_mode,
                                   shuffle=shuffle, 
                                   batch_size=batch_size)

def fit_helper(model, result_dir_name, lr=0.1, nb_epoch=1):
    K.set_value(model.optimizer.lr, lr)
    now = datetime.now().strftime("%Y%m%d_%H%M%S.h5")
    results_dir = RESULTS_DIR + "/" + result_dir_name + "/"
    distutils.dir_util.mkpath(results_dir)
    model.fit_generator(batches,
                        samples_per_epoch=batches.nb_sample,
                        nb_epoch=nb_epoch,
                        validation_data=val_batches,
                        nb_val_samples=val_batches.nb_sample,
                        callbacks=[CSVLogger(results_dir+"epoch_results.csv", separator=',', append=True)])
    model_v1.save_weights(results_dir + now )
    return model

In [4]:
batch_size = 32 # multiple of 2 and as large as you can (encourages more stable training), but no larger than 64.
batches = get_batches(train_dir, batch_size=batch_size)
val_batches = get_batches(valid_dir, batch_size=batch_size*2)

Found 200 images belonging to 2 classes.
Found 50 images belonging to 2 classes.


In [5]:
# Create model_v1
vgg_base = VGG16()
vgg_base.layers.pop()
for layer in vgg_base.layers: layer.trainable=False
layers_classifier = Dense(batches.nb_class, activation='softmax')(vgg_base.layers[-1].output)
       
model_v1 = Model(vgg_base.input, layers_classifier)
model_v1.compile(loss='categorical_crossentropy', optimizer=Adam(), metrics=['accuracy'])

>"Intially overfit befor worrying about reducing it (helping to ensure you have a model complex enough for the orignal data)."

In [6]:
fit_helper(model=model_v1, result_dir_name="model_v1", lr=0.1, nb_epoch=1)

Epoch 1/1
200/200 [==============================] - 14s - loss: 1.0488 - acc: 0.8550 - val_loss: 1.2896 - val_acc: 0.9200


In [7]:
fit_helper(model=model_v1, result_dir_name="model_v1", lr=0.1, nb_epoch=9)

Epoch 1/9
200/200 [==============================] - 5s - loss: 1.0873 - acc: 0.9300 - val_loss: 1.2894 - val_acc: 0.9200
Epoch 2/9
200/200 [==============================] - 4s - loss: 0.7642 - acc: 0.9500 - val_loss: 0.9671 - val_acc: 0.9400

In [8]:
fit_helper(model=model_v1, result_dir_name="model_v1", lr=0.1, nb_epoch=9)

Epoch 1/9
200/200 [==============================] - 5s - loss: 1.3700 - acc: 0.9150 - val_loss: 2.5789 - val_acc: 0.8400
Epoch 2/9
200/200 [==============================] - 4s - loss: 0.5252 - acc: 0.9600 - val_loss: 0.6447 - val_acc: 0.9600

In [9]:
# Appears to have saturated, try lowering learning rate
fit_helper(model=model_v1, result_dir_name="model_v1", lr=0.01, nb_epoch=5)

Epoch 1/5
200/200 [==============================] - 6s - loss: 0.4030 - acc: 0.9750 - val_loss: 0.9671 - val_acc: 0.9400
Epoch 2/5
200/200 [==============================] - 4s - loss: 0.3224 - acc: 0.9800 - val_loss: 0.9671 - val_acc: 0.9400

In [10]:
fit_helper(model=model_v1, result_dir_name="model_v1", lr=0.001, nb_epoch=5)

Epoch 1/5
200/200 [==============================] - 5s - loss: 0.5641 - acc: 0.9650 - val_loss: 0.9671 - val_acc: 0.9400
Epoch 2/5
200/200 [==============================] - 4s - loss: 0.3224 - acc: 0.9800 - val_loss: 0.9671 - val_acc: 0.9400

In [11]:
fit_helper(model=model_v1, result_dir_name="model_v1", lr=0.1, nb_epoch=5)

Epoch 1/5
200/200 [==============================] - 5s - loss: 0.3224 - acc: 0.9800 - val_loss: 0.9671 - val_acc: 0.9400
Epoch 2/5
200/200 [==============================] - 4s - loss: 0.4030 - acc: 0.9750 - val_loss: 0.9671 - val_acc: 0.9400

# 3. Generate and Validate Predictions